In [4]:
!pip install PyMuPDF

In [5]:
import fitz
import glob
import pandas as pd
import re
from operator import itemgetter
from itertools import groupby

In [7]:
broker_pdf_list = glob.glob('C:/Users/i26184/Desktop/Broker/Broker_PDF/*.pdf')

In [10]:
broker_pdf_list

['C:\\Users\\i26184\\Desktop\\Broker\\Broker_PDF\\individual_816404.pdf',
 'C:\\Users\\i26184\\Desktop\\Broker\\Broker_PDF\\individual_816436.pdf',
 'C:\\Users\\i26184\\Desktop\\Broker\\Broker_PDF\\individual_816439.pdf',
 'C:\\Users\\i26184\\Desktop\\Broker\\Broker_PDF\\individual_816452.pdf',
 'C:\\Users\\i26184\\Desktop\\Broker\\Broker_PDF\\individual_816461.pdf',
 'C:\\Users\\i26184\\Desktop\\Broker\\Broker_PDF\\individual_816467.pdf',
 'C:\\Users\\i26184\\Desktop\\Broker\\Broker_PDF\\individual_816472.pdf',
 'C:\\Users\\i26184\\Desktop\\Broker\\Broker_PDF\\individual_816473.pdf',
 'C:\\Users\\i26184\\Desktop\\Broker\\Broker_PDF\\individual_816494.pdf',
 'C:\\Users\\i26184\\Desktop\\Broker\\Broker_PDF\\individual_816500.pdf',
 'C:\\Users\\i26184\\Desktop\\Broker\\Broker_PDF\\individual_816501.pdf',
 'C:\\Users\\i26184\\Desktop\\Broker\\Broker_PDF\\individual_816506.pdf',
 'C:\\Users\\i26184\\Desktop\\Broker\\Broker_PDF\\individual_816507.pdf',
 'C:\\Users\\i26184\\Desktop\\Broker\\

In [9]:
broker_pdf_list = [x.replace('/','\\') for x in broker_pdf_list]

In [33]:
def get_broker_name_CRD(fitz_doc):
    page1 = fitz_doc.loadPage(0)
    page1_textblock = page1.getTextBlocks()
    page1_textblock = [x[4] for x in page1_textblock]
    broker_name_candiate = page1_textblock[0].split('\n')
    
    if len(broker_name_candiate)==1:
        broker_name = page1_textblock[1]
    else:
        broker_name = broker_name_candiate[1]
        
    section_start_index = [i for i in range(len(page1_textblock)) if page1_textblock[i]=='Section Title']
    section_end_index = [i for i in range(len(page1_textblock)) if 'CRD# ' in page1_textblock[i]]
    
    if len(section_end_index) == 0:
        CRD_number = 'NO CRD Number'
    else:
        CRD_number = page1_textblock[section_end_index[0]].split()[-1]
        
    return pd.DataFrame({'CRD Number':[CRD_number],
                         'Broker Name':[broker_name]})

In [120]:
def get_section_info(fitz_doc):
    page1 = fitz_doc.loadPage(0)
    page1_textblock = page1.getTextBlocks()
    page1_textblock = [x[4] for x in page1_textblock]
    
    section_start_index = [i for i in range(len(page1_textblock)) if page1_textblock[i]=='Section Title']
    section_end_index = [i for i in range(len(page1_textblock)) if 'CRD# ' in page1_textblock[i]]
    
    if len(section_end_index) == 0:
        CRD_number = 'NO CRD Number'
    else:
        CRD_number = page1_textblock[section_end_index[0]].split()[-1]
    
    if len(section_start_index)==0:
        return pd.DataFrame({'CRD Number':[CRD_number],
                             'Section Titles':['No Section'],
                             'Section Pages':['No Section']})
    section_titles = page1_textblock[(section_start_index[0]+1):section_end_index[0]]
    
    section_pages = page1_textblock[(section_end_index[0]+1):(section_end_index[0]+len(section_titles)+1)]
    
    return pd.DataFrame({'CRD Number':[CRD_number]*len(section_titles),
                         'Section Titles':section_titles,
                         'Section Pages':section_pages})

In [143]:
doc = fitz.open('C:\\Users\\i26184\\Desktop\\Broker\\Broker_PDF\\individual_816727.pdf')

In [152]:
def get_branch_office(fitz_doc):
    page1 = fitz_doc.loadPage(0)
    page1_textblock = page1.getTextBlocks()
    page1_textblock = [x[4] for x in page1_textblock]
    
    section_start_index = [i for i in range(len(page1_textblock)) if page1_textblock[i]=='Section Title']
    section_end_index = [i for i in range(len(page1_textblock)) if 'CRD# ' in page1_textblock[i]]
    
    if len(section_end_index) == 0:
        CRD_number = 'NO CRD Number'
    else:
        CRD_number = page1_textblock[section_end_index[0]].split()[-1]
    
    page_summary = fitz_doc.loadPage(2)
    page_summary_textblock = page_summary.getTextBlocks()
    page_summary_textblock = [x[4] for x in page_summary_textblock]
    
    if (any(['This broker is not currently registered' in x for x in page_summary_textblock]) or
        any(['This broker is not currently registered with' in x and 'FINRA firm' in x for x in page_summary_textblock])):
        Firm_CRD = 'NOT FOUND'
        Start_Date = 'NOT FOUND'
        Company_Info = 'NOT FOUND'
    elif any(['Currently employed by and registered with' in x for x in page_summary_textblock]):
        cutting_index = [i for i,x in enumerate(page_summary_textblock) if 'Currently employed by and registered with' in x][0]
        Firm_CRD = re.findall(r'(\d+)',[x for x in page_summary_textblock[cutting_index+1].split('\n') if 'CRD' in x][0])[0]
        Start_Date = re.findall(r'(\d+/\d+/\d+)',page_summary_textblock[cutting_index+1].split('CRD#')[-1])
        if len(Start_Date)==0:
            Start_Date = 'NOT FOUND'
        else:
            Start_Date = Start_Date[0]
        Company_Info = page_summary_textblock[cutting_index+1].split('CRD#')[0].replace('\n',' ').strip()
    return pd.DataFrame({'CRD Number':[CRD_number],
                         'Company Info':[Company_Info],
                         'Company CRD':[Firm_CRD],
                         'Date Since':[Start_Date]})
    print(page_summary_textblock)
    

In [69]:
section_info_df = pd.DataFrame()

for file in broker_pdf_list:
    print(file)
    doc = fitz.open(file)
    section_info_df = pd.concat([section_info_df,get_section_info(doc)])

/home/ubuntu/Broker/individual_816797.pdf
/home/ubuntu/Broker/individual_816523.pdf
/home/ubuntu/Broker/individual_816762.pdf
/home/ubuntu/Broker/individual_816665.pdf
/home/ubuntu/Broker/individual_816782.pdf
/home/ubuntu/Broker/individual_816511.pdf
/home/ubuntu/Broker/individual_816659.pdf
/home/ubuntu/Broker/individual_816574.pdf
/home/ubuntu/Broker/individual_816579.pdf
/home/ubuntu/Broker/individual_816707.pdf
/home/ubuntu/Broker/individual_816602.pdf
/home/ubuntu/Broker/individual_816691.pdf
/home/ubuntu/Broker/individual_816759.pdf
/home/ubuntu/Broker/individual_816517.pdf
/home/ubuntu/Broker/individual_816737.pdf
/home/ubuntu/Broker/individual_816645.pdf
/home/ubuntu/Broker/individual_816461.pdf
/home/ubuntu/Broker/individual_816696.pdf
/home/ubuntu/Broker/individual_816731.pdf
/home/ubuntu/Broker/individual_816559.pdf
/home/ubuntu/Broker/individual_816529.pdf
/home/ubuntu/Broker/individual_816704.pdf
/home/ubuntu/Broker/individual_816561.pdf
/home/ubuntu/Broker/individual_816

In [71]:
section_info_df.head(50)

,CRD Number,Section Titles,Section Pages
0,816797,Report Summary,1
1,816797,Broker Qualifications,2 - 5
2,816797,Registration and Employment History,6
0,816523,Report Summary,1
1,816523,Broker Qualifications,2 - 3
2,816523,Registration and Employment History,4 - 5
3,816523,Disclosure Events,6
0,816762,Report Summary,1
1,816762,Broker Qualifications,2 - 3
2,816762,Registration and Employment History,4


In [59]:
def separate_consecutive_numbers(num_list):
    group_list = []
    for k, g in groupby([(x, i-x) for i,x in enumerate(num_list)], lambda x:x[1]):
        group_list.append(list(g))
    group_list = [[(c[0],i) for c in x] for i,x in enumerate(group_list)]
    return group_list

In [60]:
def get_broker_qual_employ_from_pagetextblock(page_textblock):
    page_textblock = [x[4] for x in page_textblock]
    
    page_block_dated = [(i,x) for i,x in enumerate(page_textblock) if len(re.sub(r'(\d+/\d+/\d+)','',x))!=len(x)]
    
    grouped_index_block_dated = separate_consecutive_numbers([x[0] for x in page_block_dated])
    
    #print(grouped_index_block_dated)
    broker_qual_employ = pd.DataFrame()
    for group_index in grouped_index_block_dated:
        index_temp = [x[0] for x in group_index]
        table_title = ' '.join(page_textblock[(min(index_temp)-2):min(index_temp)])
        #print(table_title)
        if all([c in table_title for c in ['SRO','Category','Date']]):
            #print('SRO')
            page_textblock_temp = [page_textblock[i] for i in index_temp]
            
            SRO = [x.split('\n')[0] for x in page_textblock_temp]
            Date = [re.findall(r'(\d+/\d+/\d+)',x)[0] for x in page_textblock_temp]
            
            page_textblock_temp = [re.sub(r'(\d+/\d+/\d+)','',x).split('\n')[1].strip() for x in page_textblock_temp]
            
            Status = [' '.join([c for c in x.split() if c.upper()==c]) for x in page_textblock_temp]
            Category = [' '.join([c for c in x.split() if c.capitalize()==c]) for x in page_textblock_temp]
            
            broker_qual_employ_temp = pd.DataFrame({'Table':'SRO',
                                               'SRO or US Territory':SRO,
                                               'Category':Category,
                                               'Status':Status,
                                               'Date':Date})
            
        elif all([c in table_title for c in ['U.S. State','Territory','Category','Date']]):
            #print('US')
            page_textblock_temp = [page_textblock[i] for i in index_temp]
            
            US_Territory = [' '.join(x.split('\n')[0].split()[:-1]) for x in page_textblock_temp]
            Date = [re.findall(r'(\d+/\d+/\d+)',x)[0] for x in page_textblock_temp]
            Category = [x.split('\n')[0].split()[-1] for x in page_textblock_temp]
            page_textblock_temp = [re.sub(r'(\d+/\d+/\d+)','',x).split('\n')[1].strip() for x in page_textblock_temp]
            
            Status = [' '.join([c for c in x.split() if c.upper()==c]) for x in page_textblock_temp]
            
            
            broker_qual_employ_temp = pd.DataFrame({'Table':'US Territory',
                                               'SRO or US Territory':US_Territory,
                                               'Category':Category,
                                               'Status':Status,
                                               'Date':Date})
        else:
            continue
        
        broker_qual_employ = pd.concat([broker_qual_employ,broker_qual_employ_temp])
    
    if broker_qual_employ.shape[0]==0:
        broker_qual_employ = pd.DataFrame({'Table':['NOT FOUND'],
                                            'SRO or US Territory':['NOT FOUND'],
                                            'Category':['NOT FOUND'],
                                            'Status':['NOT FOUND'],
                                            'Date':['NOT FOUND']})
    
    return broker_qual_employ

In [61]:
def exam_class_identify_old(x):
    if all([c in x.lower() for c in ['principal','supervisory','exam']]):
        return 'Principal Supervisory Exam'
    elif all([c in x.lower() for c in ['general','industry','product','exam']]):
        return "General Industry Product Exam"
    else:
        return 'State Securities Law Exam'

In [112]:
def exam_class_identify_new(each_exam_group):
    x = ' '.join(each_exam_group)
    if all([c in x.lower() for c in ['principal','supervisory','exam']]):
        return 'Principal Supervisory Exam'
    elif all([c in x.lower() for c in ['general','industry','product','exam']]):
        return "General Industry Product Exam"
    else:
        return 'State Securities Law Exam'

In [63]:
def exam_name_identify_new(each_exam_group,i):
    if 'Examination' in each_exam_group[i]:
        return each_exam_group[i].split('Examination')[0] + ' Examination'
    else:
        x = ' '.join([each_exam_group[i-1], each_exam_group[i]])
        return x.split('Examination')[0] + ' Examination'

In [113]:
def get_broker_qual_exam_from_pagetextblock(page_textblock):
    page_textblock = [x[4] for x in page_textblock]
    
    exam_section_index = [i for i,x in enumerate(page_textblock) if all([c in x.lower() for c in ['this individual has passed']])]
    
    broker_qual_exam = pd.DataFrame()
    
    if len(exam_section_index)!=0:
        exam_count = re.findall(r'[0-9]',page_textblock[exam_section_index[0]])
        
        principal_supervisory_exam = exam_count[0]
        general_industry_product_exam = exam_count[1]
        state_securities_law_exam = exam_count[2]
        
        page_textblock = page_textblock[exam_section_index[0]:]

        exam_index = [i for i,x in enumerate(page_textblock) if all([c in x for c in ['Exam','Category','Date']])]

        exam_group = []
        
        for i,j in zip(exam_index,exam_index[1:]+[len(page_textblock)]):
            exam_group.append(page_textblock[i:j])
        #print(exam_group)
        
        
        
        for each_exam_group in exam_group:
            index_dated = [(i,x) for i,x in enumerate(each_exam_group) if len(re.sub(r'(\d+/\d+/\d+)','',x))!=len(x)]
            #ach_exam_group_dated = each_exam_group[index_dated]
            if len(index_dated)==0:
                
                continue
            elif any([True if all([c in x for c in ['Class','Exam','Category','Date']]) else False for x in each_exam_group]):
                Date = [re.findall(r'(\d+/\d+/\d+)',each_exam_group[i])[0] for i,text in index_dated]
                Exam_Class = [exam_class_identify_old(each_exam_group[i]) for i,text in index_dated] 
                Exam_Name = [each_exam_group[i].split('Exams')[-1].split('Examination')[0]+' Examination' for i,text in index_dated]    
                Category = [re.sub(r'(\d+/\d+/\d+)','',each_exam_group[i].split('Examination')[-1].strip()).strip() for i,text in index_dated]
            else:
                Date = [re.findall(r'(\d+/\d+/\d+)',each_exam_group[i])[0] for i,text in index_dated]
                Exam_Class = [exam_class_identify_new(each_exam_group) for i,text in index_dated] 
                Exam_Name = [exam_name_identify_new(each_exam_group,i) for i,text in index_dated]    
                Category = [re.sub(r'(\d+/\d+/\d+)','CUTTING_POINT',each_exam_group[i]).split('CUTTING_POINT')[-1].strip() for i,text in index_dated]
            
            broker_qual_exam_temp = pd.DataFrame({'Exam Class':Exam_Class,
                                                  'Exam Name':Exam_Name,
                                                  'Category':Category,
                                                  'Date':Date})
            
            broker_qual_exam = pd.concat([broker_qual_exam,broker_qual_exam_temp])
        
    else:
        broker_qual_exam = pd.DataFrame()
        
    if broker_qual_exam.shape[0]==0:
        Date = ['NOT FOUND']
        Exam_Class = ['NOT FOUND']
        Exam_Name = ['NOT FOUND']
        Category = ['NOT FOUND']
        broker_qual_exam = pd.DataFrame({'Exam Class':Exam_Class,
                                                  'Exam Name':Exam_Name,
                                                  'Category':Category,
                                                  'Date':Date})
    return broker_qual_exam

In [65]:
def get_regist_hist_from_pagetextblock(page_textblock):
    page_textblock = [x[4] for x in page_textblock]
    
    regist_hist_section_index = [i for i,x in enumerate(page_textblock) if all([c in x.lower() for c in ['Registration History']])]
    employ_hist_section_index = [i for i,x in enumerate(page_textblock) if all([c in x.lower() for c in ['Employment History']])]
    
    if len(regist_hist_section_index)!=0:
        if len(employ_hist_section_index)==0:
            employ_hist_section_index = [len(page_textblock)]
    
        page_textblock = page_textblock[regist_hist_section_index[0]:employ_hist_section_index[0]]
        
        index_dated = [(i,x) for i,x in enumerate(page_textblock) if len(re.sub(r'(\d+/\d+)','',x))!=len(x)]
        
        Start_Date = [re.findall(r'(\d+/\d+)',x)[0] for i,x in index_dated]
        End_Date = [re.findall(r'(\d+/\d+)',x)[1] for i,x in index_dated]
        
    else:
        registration_history = pd.DataFrame()
        
    return registration_history

In [ ]:
individual_816752

In [ ]:
individual_816782

In [ ]:
# 816782

In [36]:
broker_name_df = pd.DataFrame()
for file in broker_pdf_list:
    #print(file)
    doc = fitz.open(file)
    broker_name_df = pd.concat([broker_name_df,get_broker_name_CRD(doc)])

In [40]:
broker_name_df.tail(50)

,CRD Number,Broker Name
0,816690,LEE REGINALD COLE
0,816691,MARY ANNA GABLE
0,816693,THOMAS EDWARD YOVINE
0,816695,WING CHIN
0,816696,ELLEN M GABRIEL
0,816697,RONALD BRUCE BUCHANAN
0,816698,RICHARD CARL LAWSON
0,816699,Kenneth Ray McDonald
0,816700,MONROE IVAN POPS
0,816704,DENNIS MELVIN SCOUTEN


In [87]:
doc = fitz.open('C:\\Users\\i26184\\Desktop\\Broker\\Broker_PDF\\individual_816762.pdf')

In [87]:
doc = fitz.open('C:\\Users\\i26184\\Desktop\\Broker\\Broker_PDF\\individual_816762.pdf')

In [88]:
page1 = doc.loadPage(4).getTextBlocks()

In [115]:
page_textblock = page1

In [116]:
[x[4] for x in page_textblock]

['www.finra.org/brokercheck\nUser Guidance',
 'Broker Qualifications',
 'Industry Exams this Broker has Passed',
 'This individual has passed 0 principal/supervisory exams, 3 general industry/product exams, and 1 state\nsecurities law exam.',
 'This section includes all securities industry exams that the broker has passed. Under limited circumstances, a broker\nmay attain a registration after receiving an exam waiver based on exams the broker has passed and/or qualifying work\nexperience. Any exam waivers that the broker has received are not included below.',
 'Exam Category Date',
 'Principal/Supervisory Exams',
 'No information reported.',
 'Exam Category Date',
 'General Industry/Product Exams',
 'Securities Industry Essentials Examination 09/26/2017 SIE',
 'AMEX Put and Call Exam 06/01/1978 PC',
 'General Securities Representative Examination 12/20/1975 Series 7',
 'Exam Category Date',
 'State Securities Law Exams',
 'Uniform Securities Agent State Law Examination 05/09/1980 Serie

In [117]:
get_broker_qual_exam_from_pagetextblock(page_textblock)

,Exam Class,Exam Name,Category,Date
0,General Industry Product Exam,Securities Industry Essentials Examination,SIE,09/26/2017
1,General Industry Product Exam,Securities Industry Essentials Examination,PC,06/01/1978
2,General Industry Product Exam,General Securities Representative Examination,Series 7,12/20/1975
0,State Securities Law Exam,Uniform Securities Agent State Law Examination,Series 63,05/09/1980


In [416]:
text = 'This individual has passed 0 principal/supervisory exams, 1 general industry/product exams, and 1 state securities law exam.'

In [417]:
re.findall(r'[0-9]',text)

['0', '1', '1']

In [157]:
broker_qual_employ = pd.DataFrame()
broker_qual_exam = pd.DataFrame()
broker_name_CRD = pd.DataFrame()
broker_branch_office = pd.DataFrame()
for file in broker_pdf_list:
    print(file)
    doc = fitz.open(file)
    section_info_df_temp =  get_section_info(doc)
    #print(section_info_df_temp)
    CRD_number = section_info_df_temp.loc[0,'CRD Number']
        
    if section_info_df_temp['Section Titles'].iloc[0]!='No Section':
        broker_qualification_page = section_info_df_temp[section_info_df_temp['Section Titles']=='Broker Qualifications']['Section Pages'].iloc[0]
        pages = [int(x) for x in broker_qualification_page.split('-')]

        borker_page_textblock = []
        for page in range(min(pages),max(pages)+1):
            borker_page_textblock.extend(doc.loadPage(page+1).getTextBlocks())
    else:
        borker_page_textblock = []
        for page in range(doc.pageCount):
            if any(['Broker Qualifications' in c[4] for c in doc.loadPage(page).getTextBlocks()]):
                borker_page_textblock.extend(doc.loadPage(page).getTextBlocks())
            
    broker_qual_employ_temp =  get_broker_qual_employ_from_pagetextblock(borker_page_textblock)
    broker_qual_exam_temp =  get_broker_qual_exam_from_pagetextblock(borker_page_textblock)
    broker_qual_employ_temp['CRD Number'] = [CRD_number]*broker_qual_employ_temp.shape[0]
    broker_qual_exam_temp['CRD_Number'] = [CRD_number]*broker_qual_exam_temp.shape[0]
    
    broker_qual_employ = pd.concat([broker_qual_employ,broker_qual_employ_temp])
    broker_qual_exam = pd.concat([broker_qual_exam,broker_qual_exam_temp])
    broker_name_CRD = pd.concat([broker_name_CRD,get_broker_name_CRD(doc)])
    broker_branch_office = pd.concat([broker_branch_office,get_branch_office(doc)])

C:\Users\i26184\Desktop\Broker\Broker_PDF\individual_816404.pdf
C:\Users\i26184\Desktop\Broker\Broker_PDF\individual_816436.pdf
C:\Users\i26184\Desktop\Broker\Broker_PDF\individual_816439.pdf
C:\Users\i26184\Desktop\Broker\Broker_PDF\individual_816452.pdf
C:\Users\i26184\Desktop\Broker\Broker_PDF\individual_816461.pdf
C:\Users\i26184\Desktop\Broker\Broker_PDF\individual_816467.pdf
C:\Users\i26184\Desktop\Broker\Broker_PDF\individual_816472.pdf
C:\Users\i26184\Desktop\Broker\Broker_PDF\individual_816473.pdf
C:\Users\i26184\Desktop\Broker\Broker_PDF\individual_816494.pdf
C:\Users\i26184\Desktop\Broker\Broker_PDF\individual_816500.pdf
C:\Users\i26184\Desktop\Broker\Broker_PDF\individual_816501.pdf
C:\Users\i26184\Desktop\Broker\Broker_PDF\individual_816506.pdf
C:\Users\i26184\Desktop\Broker\Broker_PDF\individual_816507.pdf
C:\Users\i26184\Desktop\Broker\Broker_PDF\individual_816510.pdf
C:\Users\i26184\Desktop\Broker\Broker_PDF\individual_816511.pdf
C:\Users\i26184\Desktop\Broker\Broker_PD

In [124]:
broker_qual_exam.to_csv('C:\\Users\\i26184\\Desktop\\Broker\\broker_qual_exam.csv',index= False)

In [125]:
broker_qual_employ.to_csv('C:\\Users\\i26184\\Desktop\\Broker\\broker_qual_employ.csv',index= False)

In [127]:
broker_name_CRD.to_csv('C:\\Users\\i26184\\Desktop\\Broker\\broker_name_CRD.csv',index= False)

In [161]:
broker_branch_office.to_csv('C:\\Users\\i26184\\Desktop\\Broker\\broker_branch_office.csv',index= False)

In [160]:
broker_branch_office[broker_branch_office['Company Info']!='NOT FOUND'].shape

(31, 4)

In [260]:
broker_qual_employ['Category'].unique()

array(['General Securities Representative',
       'General Securities Sales Supervisor', 'Agent',
       'General Securities Principal', 'Invest. Co Variable Contracts',
       'Direct Participation Programs', 'Municipal Securities Principal',
       'Securities Trader', 'Securities Trader Principal',
       'Investment Contracts Prin', 'Government Securities Principal',
       'Government Securities Representative', 'Municipal Fund',
       'Registered Options Principal'], dtype=object)

In [387]:
doc = fitz.open('/home/ubuntu/Broker/individual_816727.pdf')

In [395]:
page1 = doc.loadPage(6).getTextBlocks()

In [396]:
page1 = [x[4] for x in page1]

In [397]:
page1

['www.finra.org/brokercheck\nUser Guidance',
 'Registration and Employment History',
 'Registration History',
 'Registration Dates Firm Name CRD# Branch Location',
 'The broker previously was registered with the following firms:',
 '04/2007 - 06/2009 MORGAN STANLEY & CO. INCORPORATED 8209 AUBURN, AL',
 '03/2003 - 04/2007 MORGAN STANLEY DW INC. 7556 AUBURN, AL',
 '01/2002 - 04/2003 CITIGROUP GLOBAL MARKETS INC. 7059 NEW YORK, NY',
 '08/1983 - 01/2002 THE ROBINSON-HUMPHREY COMPANY, LLC 723 ATLANTA, GA',
 'Employment History',
 'Employment Dates Employer Name Employer Location',
 "This section provides up to 10 years of an individual broker's employment history as reported by the individual broker on\nthe most recently filed Form U4.",
 'Please note that the broker is required to provide this information only while registered with FINRA or a national\nsecurities exchange and the information is not updated via Form U4 after the broker ceases to be registered.\nTherefore, an employment end 

In [398]:
get_SRO_registration_from_pagetextblock(doc.loadPage(3).getTextBlocks())

,Table,SRO or US Territory,Category,Status,Date
0,SRO,FINRA,General Securities Representative,APPROVED,06/01/2009
1,SRO,FINRA,General Securities Sales Supervisor,APPROVED,06/01/2009
2,SRO,FINRA,Registered Options Principal,APPROVED,06/01/2009
3,SRO,NYSE American LLC,General Securities Representative,APPROVED,06/17/2011
4,SRO,NYSE American LLC,General Securities Sales Supervisor,APPROVED,10/01/2018
5,SRO,Nasdaq Stock Market,General Securities Representative,APPROVED,06/01/2009
6,SRO,Nasdaq Stock Market,General Securities Sales Supervisor,APPROVED,06/01/2009
7,SRO,Nasdaq Stock Market,Registered Options Principal,APPROVED,06/01/2009
8,SRO,New York Stock Exchange,General Securities Representative,APPROVED,06/01/2009
9,SRO,New York Stock Exchange,General Securities Sales Supervisor,APPROVED,10/01/2018


In [83]:
page1_textblock = page1.getTextBlocks()

In [84]:
page1_textblock

[(17.649999618530273,
  7.965002059936523,
  760.8265991210938,
  19.181978225708008,
  'www.finra.org/brokercheck\nUser Guidance',
  0,
  0),
 (19.225000381469727,
  29.91501235961914,
  150.57699584960938,
  47.81601333618164,
  'Broker Qualifications',
  1,
  0),
 (22.600000381469727,
  71.90560913085938,
  99.2199935913086,
  88.42960357666016,
  'Registrations',
  2,
  0),
 (22.600000381469727,
  89.9849853515625,
  545.8397216796875,
  163.55010986328125,
  'This section provides the self-regulatory organizations (SROs) and U.S. states/territories the broker is currently\nregistered and licensed with, the category of each license, and the date on which it became effective. This section also\nprovides, for every brokerage firm with which the broker is currently employed, the address of each branch where the\nbroker works.\nThis individual is currently registered with 4 SROs and is licensed in 13 U.S. states and territories through his\nor her employer.',
  3,
  0),
 (23.7250003814